In [1]:
import pandas as pd
from sklearn.datasets import load_diabetes

# 自作パッケージのインポート
import sys
sys.path.append('../')
from mypoc import *

In [ ]:
# データセットのロード
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = pd.Series(diabetes.target, name="target")

# テスト



In [ ]:
# xgboostによる学習・予測
runner = Runner('xgb1', ModelLGB, features, params_xgb)
runner.run_train_cv()
runner.run_predict_cv()
Submission.create_submission('xgb1')